In [1]:
library(tidyverse)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In [2]:
df_eff <- read_csv('effervescence.csv', col_types = 'fffnn')

In [3]:
df_eff %>% head()

Brand,Temp,Stirred,Order,Time
<fct>,<fct>,<fct>,<dbl>,<dbl>
name,6,yes,8,77.21547
name,23,yes,3,75.37855
name,40,yes,7,68.08492
store,6,yes,1,77.87371
store,23,yes,2,66.38436
store,40,yes,18,59.82388


In [4]:
lm_eff <- lm(Time ~ Brand * Temp, data = df_eff)
aov_eff <- aov(lm_eff)
anova_eff <- anova(lm_eff)

In [5]:
me_table <- as_tibble(anova_eff[,1:3])
me_table <- cbind('Source' = c('Brand', 'Temp', 'Brand*Temp', 'Residual'), 
                  me_table, 
                  'Error Term' = c("MSA", "MSB", "MSAB", "MSE"))
me_table

Source,Df,Sum Sq,Mean Sq,Error Term
<chr>,<int>,<dbl>,<dbl>,<chr>
Brand,1,342.0072,342.007154,MSA
Temp,2,1654.7366,827.368276,MSB
Brand*Temp,2,231.8519,115.925956,MSAB
Residual,42,141.1685,3.361154,MSE


In [237]:
me_table <- as_tibble(summary(aov_eff)[[1]][,1:3])
me_table <- cbind('Source' = c('Brand', 'Temp', 'Brand*Temp', 'Residual'), 
                  me_table, 
                  'Error Term' = c("MSA", "MSB", "MSAB", "MSE"))

MSA <-  me_table[1, 4]
MSB <-  me_table[2, 4]
MSAB <- me_table[3, 4]
MSE <-  me_table[4, 4]

a <- length(levels(df_eff$Brand))
b <- length(levels(df_eff$Temp))
n <- nrow(df_eff)/(a*b)

sigma_ab <- (MSAB - MSE) / n
sigma_a <- (MSA - MSAB) / (b * n)
sigma_b <- (MSB - MSAB) / (a*n)

f_scores <- c(MSA/MSAB,MSB/MSAB,MSAB/MSE, NA)

error_dof <- c(rep(tail(me_table$Df, n = 2)[1], 2), tail(me_table$Df, n = 1)[1], NA)

me_table['Error Df'] <- error_dof

me_table["F Score"] <- f_scores
f_test <- round(1 - pf(me_table['F Score'][[1]], 
                       me_table['Df'][[1]], 
                       me_table['Error Df'][[1]]),
                4)

me_table['Pr>F'] <- f_test

se_mu <- sqrt((MSA+MSB-MSAB)/(a*b*n))
dof_app <- (MSA + MSB - MSAB)^2 / (MSA^2/(a-1) + MSB^2/(b-1) + MSAB^2 / ((a-1)*(b-1)))

name_mean <- df_eff %>% filter(Brand == 'name') %>% select(Time) %>% unlist() %>% mean()
brand_fe <- df_eff %>% group_by(Brand) %>% summarise('Mean' = mean(Time), 'Effect' = Mean - name_mean)

name_mean <- df_eff %>% filter(Temp == '6') %>% select(Time) %>% unlist() %>% mean()
temp_fe <- df_eff %>% group_by(Temp) %>% summarise('Mean' = mean(Time), 'Effect' = Mean - name_mean)

knitr::kable(me_table, caption = 'Mixed Effects Models')

knitr::kable(brand_fe, caption = 'Model4: Brand is Fixed and Temperature is Random')
knitr::kable(cbind(c('Temp', 'Brand*Temp', 'Residual'), 
                   c(sigma_b, sigma_ab, MSE)), col.names = c('Cov Parm', 'Estimate'))

knitr::kable(temp_fe, caption = 'Model5: Brand is Random and Temperature is Fixed')
knitr::kable(cbind(c('Brand', 'Brand*Temp', 'Residual'), 
                   c(sigma_a, sigma_ab, MSE)), col.names = c('Cov Parm', 'Estimate'))

knitr::kable(cbind(c('Brand', 'Temp' , 'Brand*Temp', 'Residual'), 
                   c(sigma_a, sigma_b,sigma_ab, MSE)), col.names = c('Cov Parm', 'Estimate'), 
             caption = 'Model6: Brand and Temperature are Random')



Table: Mixed Effects Models

|Source     | Df|    Sum Sq|    Mean Sq|Error Term | Error Df|   F Score|   Pr>F|
|:----------|--:|---------:|----------:|:----------|--------:|---------:|------:|
|Brand      |  1|  342.0072| 342.007154|MSA        |        2|  2.950221| 0.2280|
|Temp       |  2| 1654.7366| 827.368276|MSB        |        2|  7.137041| 0.1229|
|Brand*Temp |  2|  231.8519| 115.925956|MSAB       |       42| 34.489926| 0.0000|
|Residual   | 42|  141.1685|   3.361154|MSE        |       NA|        NA|     NA|



Table: Model4: Brand is Fixed and Temperature is Random

|Brand |     Mean|    Effect|
|:-----|--------:|---------:|
|name  | 73.44276|  0.000000|
|store | 68.10416| -5.338595|



|Cov Parm   |Estimate         |
|:----------|:----------------|
|Temp       |44.4651449693106 |
|Brand*Temp |14.0706002662374 |
|Residual   |3.36115408186724 |



Table: Model5: Brand is Random and Temperature is Fixed

|Temp |     Mean|     Effect|
|:----|--------:|----------:|
|6    | 78.00561|   0.000000|
|23   | 70.69044|  -7.315177|
|40   | 63.62433| -14.381286|



|Cov Parm   |Estimate         |
|:----------|:----------------|
|Brand      |9.42004992022257 |
|Brand*Temp |14.0706002662374 |
|Residual   |3.36115408186724 |



Table: Model6: Brand and Temperature are Random

|Cov Parm   |Estimate         |
|:----------|:----------------|
|Brand      |9.42004992022257 |
|Temp       |44.4651449693106 |
|Brand*Temp |14.0706002662374 |
|Residual   |3.36115408186724 |

In [238]:
sum(c(sigma_a, sigma_b,sigma_ab, MSE))
se_mu
dof_app

[1] 71.31695

[1] 4.684748

[1] 2.381668

In [65]:
df_nostir <- df_eff %>% filter(Stirred == 'no')

df_aov <- aov(Time ~ Brand * Temp, data = df_nostir)

me_table <- as_tibble(summary(df_aov)[[1]][,1:3])
me_table <- cbind('Source' = c('Brand', 'Temp', 'Brand*Temp', 'Residual'), 
                  me_table, 
                  'Error Term' = c("MSA", "MSB", "MSAB", "MSE"))

MSA <-  me_table[1, 4]
MSB <-  me_table[2, 4]
MSAB <- me_table[3, 4]
MSE <-  me_table[4, 4]

a <- length(levels(df_eff$Brand))
b <- length(levels(df_eff$Temp))
n <- nrow(df_eff)/(a*b)

sigma_ab <- (MSAB - MSE) / n
sigma_a <- (MSA - MSAB) / (b * n)
sigma_b <- (MSB - MSAB) / (a*n)

f_scores <- c(MSA/MSAB,MSB/MSAB,MSAB/MSE, NA)

error_dof <- c(rep(tail(me_table$Df, n = 2)[1], 2), tail(me_table$Df, n = 1)[1], NA)

me_table['Error Df'] <- error_dof

me_table["F Score"] <- f_scores
f_test <- round(1 - pf(me_table['F Score'][[1]], 
                       me_table['Df'][[1]], 
                       me_table['Error Df'][[1]]),
                4)

me_table['Pr>F'] <- f_test



name_mean <- df_eff %>% filter(Brand == 'name') %>% select(Time) %>% unlist() %>% mean()
brand_fe <- df_eff %>% group_by(Brand) %>% summarise('Mean' = mean(Time), 'Effect' = Mean - name_mean)

name_mean <- df_eff %>% filter(Temp == '6') %>% select(Time) %>% unlist() %>% mean()
temp_fe <- df_eff %>% group_by(Temp) %>% summarise('Mean' = mean(Time), 'Effect' = Mean - name_mean)
    


knitr::kable(me_table, caption = 'Mixed Effects Models')

knitr::kable(brand_fe, caption = 'Model4: Brand is Fixed and Temperature is Random')
knitr::kable(cbind(c('Temp', 'Brand*Temp', 'Residual'), 
                   c(sigma_b, sigma_ab, MSE)), col.names = c('Cov Parm', 'Estimate'))

knitr::kable(temp_fe, caption = 'Model5: Brand is Random and Temperature is Fixed')
knitr::kable(cbind(c('Brand', 'Brand*Temp', 'Residual'), 
                   c(sigma_a, sigma_ab, MSE)), col.names = c('Cov Parm', 'Estimate'))

knitr::kable(cbind(c('Brand', 'Temp' , 'Brand*Temp', 'Residual'), 
                   c(sigma_a, sigma_b,sigma_ab, MSE)), col.names = c('Cov Parm', 'Estimate'), 
             caption = 'Model6: Brand and Temperature are Random')



Table: Mixed Effects Models

|Source     | Df|     Sum Sq|     Mean Sq|Error Term | Error Df|    F Score|   Pr>F|
|:----------|--:|----------:|-----------:|:----------|--------:|----------:|------:|
|Brand      |  1| 265.011691| 265.0116910|MSA        |        2|   3.296209| 0.2111|
|Temp       |  2| 838.957737| 419.4788686|MSB        |        2|   5.217468| 0.1608|
|Brand*Temp |  2| 160.797882|  80.3989410|MSAB       |       18| 172.443232| 0.0000|
|Residual   | 18|   8.392216|   0.4662343|MSE        |       NA|         NA|     NA|



Table: Model4: Brand is Fixed and Temperature is Random

|Brand |     Mean|    Effect|
|:-----|--------:|---------:|
|name  | 73.44276|  0.000000|
|store | 68.10416| -5.338595|



|Cov Parm   |Estimate          |
|:----------|:-----------------|
|Temp       |21.1924954718672  |
|Brand*Temp |9.99158834511523  |
|Residual   |0.466234250604435 |



Table: Model5: Brand is Random and Temperature is Fixed

|Temp |     Mean|     Effect|
|:----|--------:|----------:|
|6    | 78.00561|   0.000000|
|23   | 70.69044|  -7.315177|
|40   | 63.62433| -14.381286|



|Cov Parm   |Estimate          |
|:----------|:-----------------|
|Brand      |7.6921979172706   |
|Brand*Temp |9.99158834511523  |
|Residual   |0.466234250604435 |



Table: Model6: Brand and Temperature are Random

|Cov Parm   |Estimate          |
|:----------|:-----------------|
|Brand      |7.6921979172706   |
|Temp       |21.1924954718672  |
|Brand*Temp |9.99158834511523  |
|Residual   |0.466234250604435 |

In [66]:
sum(c(sigma_a, sigma_b,sigma_ab, MSE))

[1] 39.34252

In [67]:
df_stir <- df_eff %>% filter(Stirred == 'yes')

df_aov <- aov(Time ~ Brand * Temp, data = df_stir)

me_table <- as_tibble(summary(df_aov)[[1]][,1:3])
me_table <- cbind('Source' = c('Brand', 'Temp', 'Brand*Temp', 'Residual'), 
                  me_table, 
                  'Error Term' = c("MSA", "MSB", "MSAB", "MSE"))

MSA <-  me_table[1, 4]
MSB <-  me_table[2, 4]
MSAB <- me_table[3, 4]
MSE <-  me_table[4, 4]

a <- length(levels(df_eff$Brand))
b <- length(levels(df_eff$Temp))
n <- nrow(df_eff)/(a*b)

sigma_ab <- (MSAB - MSE) / n
sigma_a <- (MSA - MSAB) / (b * n)
sigma_b <- (MSB - MSAB) / (a*n)

f_scores <- c(MSA/MSAB,MSB/MSAB,MSAB/MSE, NA)

error_dof <- c(rep(tail(me_table$Df, n = 2)[1], 2), tail(me_table$Df, n = 1)[1], NA)

me_table['Error Df'] <- error_dof

me_table["F Score"] <- f_scores
f_test <- round(1 - pf(me_table['F Score'][[1]], 
                       me_table['Df'][[1]], 
                       me_table['Error Df'][[1]]),
                4)

me_table['Pr>F'] <- f_test



name_mean <- df_eff %>% filter(Brand == 'name') %>% select(Time) %>% unlist() %>% mean()
brand_fe <- df_eff %>% group_by(Brand) %>% summarise('Mean' = mean(Time), 'Effect' = Mean - name_mean)

name_mean <- df_eff %>% filter(Temp == '6') %>% select(Time) %>% unlist() %>% mean()
temp_fe <- df_eff %>% group_by(Temp) %>% summarise('Mean' = mean(Time), 'Effect' = Mean - name_mean)
    


knitr::kable(me_table, caption = 'Mixed Effects Models')

knitr::kable(brand_fe, caption = 'Model4: Brand is Fixed and Temperature is Random')
knitr::kable(cbind(c('Temp', 'Brand*Temp', 'Residual'), 
                   c(sigma_b, sigma_ab, MSE)), col.names = c('Cov Parm', 'Estimate'))

knitr::kable(temp_fe, caption = 'Model5: Brand is Random and Temperature is Fixed')
knitr::kable(cbind(c('Brand', 'Brand*Temp', 'Residual'), 
                   c(sigma_a, sigma_ab, MSE)), col.names = c('Cov Parm', 'Estimate'))

knitr::kable(cbind(c('Brand', 'Temp' , 'Brand*Temp', 'Residual'), 
                   c(sigma_a, sigma_b,sigma_ab, MSE)), col.names = c('Cov Parm', 'Estimate'), 
             caption = 'Model6: Brand and Temperature are Random')



Table: Mixed Effects Models

|Source     | Df|    Sum Sq|    Mean Sq|Error Term | Error Df|   F Score|   Pr>F|
|:----------|--:|---------:|----------:|:----------|--------:|---------:|------:|
|Brand      |  1|  97.50550|  97.505504|MSA        |        2|  2.434286| 0.2591|
|Temp       |  2| 815.90352| 407.951760|MSB        |        2| 10.184770| 0.0894|
|Brand*Temp |  2|  80.11016|  40.055078|MSAB       |       18| 21.718234| 0.0000|
|Residual   | 18|  33.19752|   1.844306|MSE        |       NA|        NA|     NA|



Table: Model4: Brand is Fixed and Temperature is Random

|Brand |     Mean|    Effect|
|:-----|--------:|---------:|
|name  | 73.44276|  0.000000|
|store | 68.10416| -5.338595|



|Cov Parm   |Estimate         |
|:----------|:----------------|
|Temp       |22.9935426086442 |
|Brand*Temp |4.77634650025858 |
|Residual   |1.84430644204827 |



Table: Model5: Brand is Random and Temperature is Fixed

|Temp |     Mean|     Effect|
|:----|--------:|----------:|
|6    | 78.00561|   0.000000|
|23   | 70.69044|  -7.315177|
|40   | 63.62433| -14.381286|



|Cov Parm   |Estimate         |
|:----------|:----------------|
|Brand      |2.393767732355   |
|Brand*Temp |4.77634650025858 |
|Residual   |1.84430644204827 |



Table: Model6: Brand and Temperature are Random

|Cov Parm   |Estimate         |
|:----------|:----------------|
|Brand      |2.393767732355   |
|Temp       |22.9935426086442 |
|Brand*Temp |4.77634650025858 |
|Residual   |1.84430644204827 |

In [68]:
sum(c(sigma_a, sigma_b,sigma_ab, MSE))

[1] 32.00796

In [69]:
library(car)
Anova(lm(Time ~ Temp + Brand * Stirred, data = df_eff), type = 3)

,Sum Sq,Df,F value,Pr(>F)
,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),49693.91326,1,7384.919898,7.765287e-49
Temp,1654.73655,2,122.953659,2.779255e-18
Brand,97.50550,1,14.490111,4.517511e-04
Stirred,83.05926,1,12.343282,1.073633e-03
Brand:Stirred,20.51004,1,3.047959,8.815069e-02
Residuals,282.62248,42,NA,NA


In [47]:
mean(df_eff$Time)
var(df_eff$Time)

[1] 70.77346

[1] 50.42051

In [57]:
df_eff %>% filter(Brand == 'name') %>% select(Time) %>% unlist() %>% unname() %>% mean()
df_eff %>% filter(Brand == 'store') %>% select(Time) %>% unlist() %>% unname() %>% mean()

[1] 73.44276

[1] 68.10416

In [61]:
df_eff %>% group_by(Brand) %>% summarise('Mean' = mean(Time), 'Var' = var(Time))
df_eff %>% group_by(Stirred) %>% summarise('Mean' = mean(Time), 'Var' = var(Time))
df_eff %>% group_by(Temp) %>% summarise('Mean' = mean(Time), 'Var' = var(Time))

Brand,Mean,Var
<fct>,<dbl>,<dbl>
name,73.44276,21.13144
store,68.10416,67.03190


Stirred,Mean,Var
<fct>,<dbl>,<dbl>
yes,69.56681,44.63986
no,71.98011,55.35476


Temp,Mean,Var
<fct>,<dbl>,<dbl>
6,78.00561,2.467426
23,70.69044,19.204515
40,63.62433,25.996561


In [204]:
means_nostir <- df_eff %>% filter(Stirred == "no") %>% group_by(Brand, Temp) %>% summarise('Mean' = mean(Time))
means_nostir <- means_nostir %>% pivot_wider(names_from = Brand, values_from = Mean)
means_nostir$TempMean <- rowMeans(means_nostir[,2:3])
means_nostir <- cbind('Temp' = c('6', '23', '40', 'BrandMean'),rbind(means_nostir[,2:4], colMeans(means_nostir[,2:4])))
means_nostir

`summarise()` has grouped output by 'Brand'. You can override using the `.groups` argument.


Temp,name,store,TempMean
<chr>,<dbl>,<dbl>,<dbl>
6,78.99061,79.49240,79.24151
23,76.36351,67.51552,71.93952
40,70.55511,58.96347,64.75929
BrandMean,75.30308,68.65713,71.98011


In [206]:
means_stir <- df_eff %>% filter(Stirred == "yes") %>% group_by(Brand, Temp) %>% summarise('Mean' = mean(Time))
means_stir <- means_stir %>% pivot_wider(names_from = Brand, values_from = Mean)
means_stir$TempMean <- rowMeans(means_stir[,2:3])
means_stir <- cbind('Temp' = c('6', '23', '40', 'BrandMean'),rbind(means_stir[,2:4], colMeans(means_stir[,2:4])))
means_stir

`summarise()` has grouped output by 'Brand'. You can override using the `.groups` argument.


Temp,name,store,TempMean
<chr>,<dbl>,<dbl>,<dbl>
6,76.20241,77.33703,76.76972
23,72.69145,66.19126,69.44135
40,65.85343,59.12529,62.48936
BrandMean,71.58243,67.55119,69.56681


In [236]:
means_table <- df_eff %>% group_by(Brand, Temp) %>% summarise('Mean' = mean(Time))
means_table <- means_table %>% pivot_wider(names_from = Brand, values_from = Mean)
means_table$TempMean <- rowMeans(means_table[,2:3])
means_table <- cbind('Temp' = c('6', '23', '40', 'BrandMean'),rbind(means_table[,2:4], colMeans(means_table[,2:4])))
knitr::kable(means_table)

`summarise()` has grouped output by 'Brand'. You can override using the `.groups` argument.




|Temp      |     name|    store| TempMean|
|:---------|--------:|--------:|--------:|
|6         | 77.59651| 78.41471| 78.00561|
|23        | 74.52748| 66.85339| 70.69044|
|40        | 68.20427| 59.04438| 63.62433|
|BrandMean | 73.44276| 68.10416| 70.77346|

In [230]:
# Random effect Analysis
re_table <- as_tibble(cbind(c('Brand', 'Temp' , 'Brand*Temp', 'Residual'), round(c(sigma_a, sigma_b,sigma_ab, MSE), 4))) 
colnames(re_table) <- c('Cov Parm', 'Estimate')
re_table$Estimate <- as.numeric(re_table$Estimate)
re_table$Portion <- round(as.numeric(re_table$Estimate) / sum(as.numeric(re_table$Estimate)),3)
re_table

Cov Parm,Estimate,Portion
<chr>,<dbl>,<dbl>
Brand,9.4200,0.132
Temp,44.4651,0.623
Brand*Temp,14.0706,0.197
Residual,3.3612,0.047


In [227]:
re_table$Estimate <- as.numeric(re_table$Estimate)
re_table$Percent <- round(as.numeric(re_table$Estimate) / sum(as.numeric(re_table$Estimate)),3)

In [228]:
re_table

Cov Parm,Estimate,Percent
<chr>,<dbl>,<dbl>
Brand,9.4200,0.132
Temp,44.4651,0.623
Brand*Temp,14.0706,0.197
Residual,3.3612,0.047


In [232]:
se_mu

[1] 4.684748

In [234]:
means_table %>% mutate('Diff' = name - store)

Temp,name,store,TempMean,Diff
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
6,77.59651,78.41471,78.00561,-0.8181992
23,74.52748,66.85339,70.69044,7.6740900
40,68.20427,59.04438,63.62433,9.1598941
BrandMean,73.44276,68.10416,70.77346,5.3385950
